# mf6adj box example

In this notebook, we will test MF6ADJ using a simple box problem and compare the result with analytical solution

In [ ]:
import os
import pathlib as pl
import platform
import shutil
import sys
from datetime import datetime

import flopy
import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyemu

try:
    import mf6adj
except ImportError:
    sys.path.insert(0, str(pl.Path("../").resolve()))
    import mf6adj

First we need to get the platform-specific binaries. We have some strict control over these and they are stored at the root level in the repo in the bin dir. Let's workout what path we should be using and the binary names we need:

In [ ]:
env_path = pl.Path(os.environ.get("CONDA_PREFIX", None))
assert env_path is not None, "Notebook must be run from the mf6adj Conda environment"

In [ ]:
bin_path = "bin"
exe_ext = ""
if "linux" in platform.platform().lower():
    lib_ext = ".so"
elif "darwin" in platform.platform().lower() or "macos" in platform.platform().lower():
    lib_ext = ".dylib"
else:
    bin_path = "Scripts"
    lib_ext = ".dll"
    exe_ext = ".exe"
lib_name = env_path / f"{bin_path}/libmf6{lib_ext}"
mf6_bin = env_path / f"{bin_path}/mf6{exe_ext}"

Now let's get the model files we will be using - they are stored in the autotest directory (here names as: xd_box_chd_ana)

In [ ]:
org_ws = os.path.join("xd_box_chd_ana")
assert os.path.exists(org_ws)

setup a local copy of the model files. Also copy in the binaries we need for later....

In [ ]:
ws = "xd_box_chd_ana_working"
if os.path.exists(ws):
    shutil.rmtree(ws)
shutil.copytree(org_ws, ws)

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_ws=ws)
m = sim.get_model()

In [ ]:
ib = m.dis.idomain.array[0, :, :].astype(float)
ib[ib > 0] = np.nan
ib_cmap = plt.get_cmap("Greys_r")
ib_cmap.set_bad(alpha=0.0)


def plot_model(arr, units=None):
    arr[~np.isnan(ib)] = np.nan
    fig, ax = plt.subplots(1, 1, figsize=(6, 6))
    cb = ax.imshow(arr, cmap="plasma")
    plt.colorbar(cb, ax=ax, label=units)
    plt.imshow(ib, cmap=ib_cmap)
    return fig, ax

Run the existing model in our local workspace


In [ ]:
print(mf6_bin)
print(ws)
print(os.listdir(ws))
pyemu.os_utils.run(mf6_bin.name, cwd=ws)

Now plot some heads...

In [ ]:
hds = flopy.utils.HeadFile(os.path.join(ws, "xdbox.hds"))
final_arr = hds.get_data()
fig, ax = plot_model(final_arr[0, :, :], units="meters")
ax.set_title("layer 1 final heads")

The main requirement to use Mf6Adj is an input file that describes the performance measures. Luckily this file has a nice modern format like other MF6 input files.


In [ ]:
pm_fname = "xdbox.dat"
fpm = open(os.path.join(ws, pm_fname), "w")

In [ ]:
l, r, c = 1, 41, 21  # the layer row and column
sp, ts = 1, 1  # stress period and time step
pm_name = "pm_single"
fpm.write("begin performance_measure {0}\n".format(pm_name))
fpm.write("{0} {1} {2} {3} {4} head direct 1.0 -1e30\n".format(sp, ts, l, r, c))
fpm.write("end performance_measure\n\n")
fpm.close()

Ok, now we should be ready to go...the adjoint solution process requires running the model forward once and then solving for the adjoint state, which uses the forward solution components (i.e. the conductance matrix, the RHS, heads, saturation,etc). The adjoint state solution has two important characteristics: its a linear (independent of the forward model's linearity) and it solves backward in time, starting with the last stress period - WAT?!

In [ ]:
bd = os.getcwd()
os.chdir(ws)

In [ ]:
forward_hdf5_name = "forward.hdf5"
start = datetime.now()
import mf6adj

adj = mf6adj.Mf6Adj(pm_fname, lib_name, logging_level="INFO")
adj.solve_gwf(hdf5_name=forward_hdf5_name)  # solve the standard forward solution
dfsum = adj.solve_adjoint()  # solve the adjoint state for each performance measure
adj.finalize()  # release components
duration = (datetime.now() - start).total_seconds()
print("took:", duration)

In [ ]:
os.chdir(bd)

Let's see what happened...

In [ ]:
[f for f in os.listdir(ws) if f.endswith("hdf5")]

MF6ADJ uses the widely available HDF5 format to store information - these files hold very low-level granular information about the adjoint solution. However the mf6adj.solve_adjoint() method also returns a higher-level summary of the adjoint solution. Let's look at it first:

In [ ]:
type(dfsum)

In [ ]:
list(dfsum.keys())

In [ ]:
dfhw = dfsum["pm_single"]
dfhw

those are the node-scale sensitivities to the sfr flux-based performance measure - some plots would be nice you say?! Well this is most easily done with the HDF5 file itself...

In [ ]:
result_hdf = "adjoint_solution_pm_single_forward.hdf5"
hdf = h5py.File(os.path.join(ws, result_hdf), "r")
keys = list(hdf.keys())
keys.sort()
print(keys)

The "composite" group has the sensitivities of the performance measure to the model inputs summed across all adjoint solutions...

In [ ]:
grp = hdf["composite"]
plot_keys = [i for i in grp.keys() if len(grp[i].shape) == 3]
plot_keys
grp["wel6_q"][:]

A simple routine to plot all these sensitivities....

In [ ]:
for pkey in plot_keys:
    arr = grp[pkey][:]
    for k, karr in enumerate(arr):
        karr[karr == 0.0] = np.nan
        fig, ax = plot_model(karr)
        ax.set_title(pkey + ", layer:{0}".format(k + 1), loc="left")

Defining the analytical solution (Lu and Vesselinov, WRR, 2015: doi:10.1002/2014WR016819) 

In [ ]:
H1, H2 = 1001.0, 1000.0  # Heads at both sides
L1, L2 = 500.0, 400.0  # Domain size
nlay, nrow, ncol = m.dis.nlay.data, m.dis.nrow.data, m.dis.ncol.data
delrow, delcol = m.dis.delr.data[0], m.dis.delc.data[0]
L1 = delrow * ncol
L2 = delcol * nrow
H = 1.0
k = 10.0  # Hydraulic conductivity
T = k * H  # Transmissivity
D = L1 * L2


def alpha(m):
    return m * np.pi / L1


def beta(n):
    return n * np.pi / L2


def omega_square(m, n):
    return (alpha(m)) ** 2 + (beta(n)) ** 2


def phi_s(x, y, xs, ys, M=5, N=5):
    listofmindices = [item + 1 for item in range(M)]
    Sum = 0.0
    a = [0.5]
    for i in [item + 1 for item in range(N)]:
        a.append(1.0)
    for n in range(N + 1):
        for m in listofmindices:
            Sum = Sum + a[n] * np.sin(alpha(m) * x) * np.cos(beta(n) * y) * np.sin(
                alpha(m) * xs
            ) * np.cos(beta(n) * ys) / omega_square(m, n)
    Sum = (4 / (T * D)) * Sum
    return Sum


def get_analytical_adj_state(xs, ys, M, N):
    list_adj_state = []
    for j in range(nrow):
        for i in range(ncol):
            x = m.modelgrid.xcellcenters[j][i]
            y = m.modelgrid.ycellcenters[j][i]
            list_adj_state.append(phi_s(x, y, xs, ys, M=M, N=N))
    array_adj_state = np.array(list_adj_state)
    print("array_adj_state = ", array_adj_state)
    filepath = os.path.join(ws, "lamb_Analytical.txt")
    with open(filepath, "w") as file:
        # with open('lamb_Analytical.txt', 'w') as file:
        for value in array_adj_state:
            file.write(f"{value}\n")
    array_adj_state_2D = np.reshape(array_adj_state, (nrow, ncol))
    return array_adj_state_2D

In [ ]:
def plot_colorbar_contour(x, y, l_anal, l_num, vmin, vmax):
    fig, axes = plt.subplots(1, 2, figsize=(14, 5), constrained_layout=True)
    ax = axes[0]
    ax.set_title("Analytical", fontsize=16)
    cs = ax.contourf(x, y, l_anal, zorder=1, vmin=vmin, vmax=vmax)
    ax.contour(cs, colors="k", linewidths=1.0, linestyles="-")
    plt.colorbar(cs)
    ax.set_xlabel("x (m)", fontsize=14)
    ax.set_ylabel("y (m)", fontsize=14)
    ax.text(-60.0, 420.0, "a", weight="bold", fontsize=18)
    plt.sca(ax)
    plt.xticks(
        [0, 100, 200, 300, 400, 500], ["0", "1000", "2000", "3000", "4000", "5000"]
    )
    plt.yticks(
        [0, 50, 100, 150, 200, 250, 300, 350, 400],
        ["0", "500", "1000", "1500", "2000", "2500", "3000", "3500", "4000"],
    )
    ax = axes[1]
    ax.set_title("MF6-ADJ", fontsize=16)
    cs = ax.contourf(x, y, l_num, zorder=1, vmin=vmin, vmax=vmax, levels=cs.levels)
    ax.contour(cs, colors="k", linewidths=1.0, linestyles="-")
    plt.colorbar(cs)
    ax.set_xlabel("x (m)", fontsize=14)
    ax.set_ylabel("y (m)", fontsize=14)
    ax.text(-60.0, 420.0, "b", weight="bold", fontsize=18)
    plt.sca(ax)
    plt.xticks(
        [0, 100, 200, 300, 400, 500], ["0", "1000", "2000", "3000", "4000", "5000"]
    )
    plt.yticks(
        [0, 50, 100, 150, 200, 250, 300, 350, 400],
        ["0", "500", "1000", "1500", "2000", "2500", "3000", "3500", "4000"],
    )

In [ ]:
# M & N should be large enough to get convergence
# of the analytical series solution (e.g., M=N=100)
lam_anal = get_analytical_adj_state(100.0, 200.0, M=100, N=100)
file_path = os.path.join(ws, "lamb_Analytical.txt")
with open(file_path, "r") as file:
    lamb_ana = [float(line.strip()) for line in file]
    lamb_ana = np.array(lamb_ana)
    lamb_ana_arr = lamb_ana
    lamb_ana = np.reshape(lamb_ana, (nrow, ncol))

In [ ]:
lam_adj = grp["wel6_q"][:]
x = np.linspace(0, L1, ncol)
y = np.linspace(0, L2, nrow)
x_center = [x[i] + delcol / 2.0 for i in range(len(x))]
y_center = [y[i] + delrow / 2.0 for i in range(len(y))]
y = y[::-1]
y_center = y_center[::-1]
vmin, vmax = lamb_ana.min(), lamb_ana.max()
contour_intervals = np.arange(vmin, vmax, 0.003)
plot_colorbar_contour(x, y, lamb_ana, lam_adj[0], vmin, vmax)